# Notebook to Segment Skeletal Muscle Area from MRI images (TIFs)

###### Load libraries and directories

In [1]:
import os
os.getcwd()
os.chdir('/tf/smipipeline')
print(os.getcwd())

/tf/smipipeline


In [2]:
from IPython.display import display, HTML

In [3]:
# from IPython import get_ipython
from tqdm.notebook import tqdm
import pickle
import os
import pprint
import numpy as np

import tables
from MRI_notebooks.TIF_loader import mri_TIF_loader
from unet3d.normalize import normalize_data_storage_2D


pp = pprint.PrettyPrinter(indent=1)
 
import json
import pprint

# Custom functions
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [4]:
cwd = os.getcwd()
print(cwd)
data = '/tf/data'
pickles = '/tf/pickles'
models = '/tf/models'

/tf/smipipeline


In [5]:
# Import modules and config file
configfile = os.path.join(cwd,'config/mri/sma_mri_prediction_fresh.json')
with open(configfile, "r") as f:
        config = json.load(f)
pp.pprint(config)

{'all_modalities': ['MR'],
 'data_file': '/tf/data/sarcopeniaMR_L3_h5/mri.h5',
 'image_masks': ['truth'],
 'input_type': 'Image',
 'labels': ['1'],
 'output_dir': '/tf/output/mri/',
 'overwrite': False,
 'prediction_model_path': '/tf/models/muscle/mri/ct_mri_invert_fresh.h5',
 'problem_type': 'Segmentation',
 'show_plots': False,
 'testing_split': '/tf/pickles/mri/test_0.2.pkl',
 'training_modalities': ['MR'],
 'training_split': '/tf/pickles/mri/train_0.7.pkl',
 'validation_split': '/tf/pickles/mri/validation_0.1.pkl'}


### Set overwrite to false when extracting imags from h5 file

In [6]:
config['overwrite'] = False

In [7]:
output_file = config['data_file']

In [8]:
# Step 1: Check if Output file already exists, If it exists, require user permission to overwrite
if 'overwrite' in config:
    overwrite = config["overwrite"]
elif os.path.exists(output_file):
    overwrite = input("Output file exists, do you want to overwrite? (y/n) \n")
    overwrite = True if overwrite == 'y' else False   

    # Open the hdf5 file
if overwrite:
    print("Opening h5 file in Write mode...")
    hdf5_file = tables.open_file(output_file, mode='w')
else:
    print("Opening h5 file in Read mode...")
    hdf5_file = tables.open_file(output_file, mode='r')

Opening h5 file in Read mode...


In [9]:
subject_ids_final = hdf5_file.root.subject_ids[:]
subject_ids_final = [subject.decode("utf-8")for subject in subject_ids_final]
print("Total number of subjects in h5 file", len(subject_ids_final))

Total number of subjects in h5 file 201


In [10]:
subject_ids_final[:5]

['exam 109c', 'exam 97b', 'exam 1', 'exam 46', 'exam 107']

In [11]:
data_file =hdf5_file

In [12]:
from PIL import Image
import numpy as np

In [13]:
pickle_file = config['testing_split']
pickle_list = load_object(pickle_file)
pickle_name = pickle_file.split('/')[-1].split('.pkl')[0]

In [14]:
config['output_directory'] = os.path.join(config['output_dir'],pickle_name)
if not os.path.exists(config['output_directory']):
    os.makedirs(config['output_directory'])
output_dir = config['output_directory']

In [15]:
config['model_path'] = config['prediction_model_path']
#config['model_path'] = config['transfer_learning_model']

In [16]:
from run_sma_experiment import configure_and_load_model
model = configure_and_load_model(config["model_path"])

In [17]:
training_modalities = config['training_modalities']
config['threshold'] = 0.5

In [18]:
from unet3d.prediction2D import *

In [19]:
for index in pickle_list:
    print(index)
    if 'subject_ids' in data_file.root:
        case_directory = os.path.join(output_dir, data_file.root.subject_ids[index].decode('utf-8'))
    else:
        case_directory = os.path.join(output_dir, "validation_case_{}".format(index))

    run_validation_case(data_index=index, output_dir=case_directory, model=model, data_file=data_file,
                                training_modalities=config["training_modalities"], output_label_map=True, labels=config["labels"],
                                threshold=config["threshold"])

92
10
65
64
100
169
82
99
147
193
154
162
3
136
143
37
80
18
56
95
7
52
36
9
98
171
125
44
93
115
189
83
8
173
21
195
192
132
191
104
166


In [24]:
hdf5_file.close()

#### Run Evaluation

In [25]:
from run_evaluation_onTIF import *

In [26]:
config_predict = config.copy()
config['input_dir'] = '/tf/output/mri/test_0.2'
config['truth_img'] = 'truth'
config['prediction_img'] = 'prediction'
config['output_file'] = 'test_dsc_freshmodel.csv'

In [27]:
header = ("DSC",)
masking_functions = (get_mask,)
rows = list()
subject_ids = list()
for case_folder in glob.glob(config['input_dir'] + "/*"):
    if not os.path.isdir(case_folder):
        continue
    subject_ids.append(os.path.basename(case_folder))
    truth_file = os.path.join(case_folder, config['truth_img']+".TIF")
    truth = io.imread(truth_file)
    prediction_file = os.path.join(case_folder, config['prediction_img']+".TIF")
    prediction = io.imread(prediction_file)
    rows.append([dice_coefficient(func(truth), func(prediction))for func in masking_functions])

df = pd.DataFrame.from_records(rows, columns=header, index=subject_ids)
print('Index of df:', df.index,'\t', df['DSC'], '\n \n')
print('Mean: ', np.mean(df['DSC']))
print('Median: ', np.median(df['DSC']))
print('Min: ', np.min(df['DSC']))
print('Max: ', np.max(df['DSC']))

df.to_csv(config['input_dir'] + '/' + config['output_file'])

scores = dict()
for index, score in enumerate(df.columns):
    values = df.values.T[index]
    scores[score] = values[np.isnan(values) == False]

plt.boxplot(list(scores.values()), labels=list(scores.keys()))
plt.ylabel("Dice Coefficient")
plt.savefig(os.path.join(config['input_dir'], "validation_scores_boxplot_test2.png"))
plt.close()

if os.path.exists("./training.log"):
    training_df = pd.read_csv("./training.log").set_index('epoch')

    plt.plot(training_df['loss'].values, label='training loss')
    plt.plot(training_df['val_loss'].values, label='validation loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.xlim((0, len(training_df.index)))
    plt.legend(loc='upper right')
    plt.savefig('loss_graph_test1_train2.png')

Index of df: Index(['exam 46', 'exam 146', 'exam 120', 'exam 152b', 'exam 144b', 'exam 142',
       'exam 119', 'exam 82', 'exam 35', 'exam 103', 'exam 118', 'exam 91',
       'exam 69', 'exam 52', 'exam 80b', 'exam 186', 'exam 65', 'exam 115',
       'exam 3', 'exam 27', 'exam 210', 'exam 202', 'exam 5', 'exam 180',
       'exam 128', 'exam 51', 'exam 36', 'exam 29', 'exam 39', 'exam 156b',
       'exam 30', 'exam 93', 'exam 84', 'exam 26', 'exam 130', 'exam 48',
       'exam 85', 'exam 106', 'exam 20', 'exam 148b', 'exam 121'],
      dtype='object') 	 exam 46      0.920182
exam 146     0.894771
exam 120     0.937284
exam 152b    0.924297
exam 144b    0.856232
exam 142     0.940682
exam 119     0.946655
exam 82      0.934533
exam 35      0.934944
exam 103     0.907533
exam 118     0.921734
exam 91      0.935496
exam 69      0.837310
exam 52      0.947126
exam 80b     0.915924
exam 186     0.908022
exam 65      0.932910
exam 115     0.931034
exam 3       0.924910
exam 27      0.914424
